In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import csv
import os

In [2]:
data = []
file1 = 'train.csv'
file2 = 'test.csv'

# the height and width of our final image
fnl_hyt = 32
fnl_wdt = 32
pad_value = 1

In [3]:
if(os.path.exists(file1) and os.path.isfile(file1)): 
  os.remove(file1) 
if(os.path.exists(file2) and os.path.isfile(file2)): 
  os.remove(file2) 

In [4]:
def crop_cond(arr):
    if np.min(arr) == 0 :
        return True
    else:
        return False

In [5]:
def crop(image):
    img_wdt = len(image[0])
    img_hyt = len(image)
    non_white_pixels = np.argwhere(image != pad_value)

    top_mrgn = 0
    bot_mrgn = img_hyt-1
    left_mrgn = 0
    right_mrgn = img_wdt-1

    # method 1****************************************************

    top_mrgn, left_mrgn = non_white_pixels.min(axis=0)
    bot_mrgn, right_mrgn = non_white_pixels.max(axis=0)

    # method 2*****************************************************
    # for i in range(img_hyt):
    #     if crop_cond(image[i]):
    #         top_mrgn = i
    #         break
    
    # for i in range(img_hyt):
    #     if crop_cond(image[img_hyt - i - 1]):
    #         bot_mrgn = img_hyt - i - 1
    #         break
    
    # t_image = image.T
    # for i in range(img_wdt):
    #     if crop_cond(t_image[i]):
    #         left_mrgn = i
    #         break
    
    # for i in range(img_wdt):
    #     if crop_cond(t_image[img_wdt - i - 1]):
    #         bot_mrgn = img_wdt - i - 1
    #         break
    
    return image[top_mrgn:bot_mrgn+1 , left_mrgn:right_mrgn+1]

In [6]:
# the value which we are padding

def pad(image):
    img_hyt = len(image)
    img_wdt = len(image[0])

    pad_hyt = img_hyt
    pad_wdt = img_wdt

    while(pad_hyt%fnl_hyt != 0):
        pad_hyt += 1

    while(pad_wdt%fnl_wdt != 0):
        pad_wdt += 1

    if pad_wdt//fnl_wdt > pad_hyt//fnl_hyt:
        pad_hyt = fnl_hyt*(pad_wdt//fnl_wdt)
    else :
        pad_wdt = fnl_wdt*(pad_hyt//fnl_hyt)
    
    top_pad = np.zeros(((pad_hyt-img_hyt)//2, img_wdt)) + pad_value
    bot_pad = np.zeros(((pad_hyt-img_hyt+1)//2 , img_wdt)) + pad_value
    left_pad = np.zeros((pad_hyt, (pad_wdt-img_wdt)//2)) + pad_value
    right_pad = np.zeros((pad_hyt, (pad_wdt-img_wdt+1)//2)) + pad_value

    # Stack the padding arrays and the image
    return np.hstack((left_pad, np.vstack((top_pad, image,bot_pad)) ,right_pad))

In [7]:
from scipy.ndimage import zoom  
def reshape(image):
    return zoom(image,(fnl_hyt/len(image), fnl_wdt/len(image[0])) , order = 1)

In [8]:
images = {}
names = {}

for i in range(1,63):
    for j in range(1,56):
        name = "img0"
        if i < 10:
            name += "0" + str(i) + "-0"
        else:
            name += str(i) + "-0"
        if j < 10:
            name += "0" + str(j) + ".png"
        else:
            name += str(j) + ".png"
        image = Image.open("Img/" + name)
        
        print(type(image))
        
        # Load, resize, and extract pixel values for each image
        if image is not None:
            # Convert the image to grayscale if it's in color
            if image.mode != "L":
                image = image.convert("L")
                threshold = 127
                image = image.point(lambda pixel: 1 if pixel > threshold else 0, '1')

            image = np.array(image)
        else:
            print("Image not found or couldn't be loaded")
        images[i*100 + j] = image
        names[i*100 + j] = name

<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImageFile'>
<class 'PIL.PngImagePlugin.PngImag

In [9]:

def trim(image, avg):
    int_fill = 1 - (np.sum(image) / (image.shape[0] * image.shape[1]))
    fill = int_fill

    if fill > avg :
        while fill > avg:
            change = np.argwhere((image - np.roll(image, 1, axis=0)) +
                                (image - np.roll(image, 1, axis=1)) +
                                (image - np.roll(image, -1, axis=0)) +
                                (image - np.roll(image, -1, axis=1)) < 0)

            image[change[:, 0], change[:, 1]] = pad_value
            tfill = 1 - (np.sum(image) / (image.shape[0] * image.shape[1]))

            if tfill >= fill:
                raise ValueError("Error: Unable to trim the image further.")

            fill = tfill
    
    else :
        image = 1 - image
        fill = 1 - fill
        while fill >= 1 - avg:
            change = np.argwhere((image - np.roll(image, 1, axis=0)) +
                                (image - np.roll(image, 1, axis=1)) +
                                (image - np.roll(image, -1, axis=0)) +
                                (image - np.roll(image, -1, axis=1)) < 0)

            image[change[:, 0], change[:, 1]] = pad_value
            tfill = 1 - (np.sum(image) / (image.shape[0] * image.shape[1]))

            if tfill >= fill:
                raise ValueError("Error: Unable to trim the image further.")

            fill = tfill
        image = 1 - image

    return image


In [10]:
def transform(image):
    return reshape(pad(crop(image)))
    # return crop(image)

In [11]:
indexs = list(images.keys())

processed_images = {}
for i in indexs:
    processed_images[i] = transform(images[i])

fill_avg = []
for i in indexs:
    processed_images[i][processed_images[i] >= 0.5] = 1
    processed_images[i][processed_images[i] < 0.5] = 0
    fill_avg.append(1- (np.sum(processed_images[i])/(processed_images[i].shape[0]*processed_images[i].shape[1])))
    
avg = np.mean(np.array(fill_avg))
print(avg)

# for i in indexs:
#     processed_images[i] = trim(processed_images[i], 0.1)


0.2832824528042522


In [12]:
for i in indexs:
    Image.fromarray(processed_images[i]*255).convert('RGB').save("./image_data_processed/" + names[i][:-4]+".png")

In [13]:
with open("data.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([i for i in range(fnl_hyt*fnl_wdt + 1)])
    for i in indexs:
        writer.writerow([i//100 - 1] + list(processed_images[i].flatten()))

In [ ]:
idx = np.random.choice(np.array(range(55)),40,replace=False)
idx.sort()
with open("train.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([i for i in range(fnl_hyt*fnl_wdt + 1)])
    for j in range(10, 62):
        for i in idx:
            writer.writerow([j - 10] + list(processed_images[j*100 + i+101].flatten()))

In [ ]:
with open("test.csv", mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([i for i in range(fnl_hyt*fnl_wdt + 1)])
    for j in range(10, 62):
        for i in range(0,55):
            if i not in idx:
                writer.writerow([j - 10] + list(processed_images[j*100 + i+101].flatten()))